In [1]:
import librosa
import soundfile
import os, glob, pickle
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

c:\users\jonma\programming\anaconda\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
c:\users\jonma\programming\anaconda\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
c:\users\jonma\programming\anaconda\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


In [ ]:
# Extract info from audio files
def extract_audio_features(file_name, mfcc, chroma, mel):
    """
    mfcc represents the short term power spectrum of the sound
    chroma is the pitch
    mel is the spectrogram frequency
    """
    with soundfile.SoundFile(file_name) as sound_file:
        X = sound_file.read(dtype="float32")
        sample_rate = sound_file.sample_rate
        if chroma:
            matrix_freq = np.abs(librosa.stft(X))
        result = np.array([])

        if mfcc:
            pwr_spec = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40))
            result = np.hstack((result, pwr_spec)) # add to result
            